### A Song of Vice and Higher: Characterizing Presidential Nominees through Game of Thrones

### Import Necessary Libraries

In [1]:
import os             # file system stuff
import json           # digest json
import pandas as pd   # Dataframes
import helper     # Custom helper functions
import gensim 

### Load the database keys

3. Create your first authorized Reddit instance.

In [22]:
# Define path to secret
secret_path = os.path.join(os.environ['HOME'], 'mia/.secret', 'reddit_api.json')

In [23]:
keys = helper.get_keys(secret_path)

In [24]:
reddit = praw.Reddit(client_id=keys['client_id'] 
                     ,client_secret=keys['api_key']
                     ,username=keys['username']
                     ,password=keys['password']
                     ,user_agent='reddit_research accessAPI:v0.0.1 (by /u/FlatDubs)')

4. Obtain a Subreddit Instance from your Reddit Instance. Ours will come from two different subreddits.

In [25]:
politics = reddit.subreddit('politics')

In [26]:
got = reddit.subreddit('gameofthrones')

5. Obtain a submission instance from your Subreddit instance and compile the submission stats to a list
6. Create a Pandas dataframe of the submission stats

In [27]:
#step 5 obtain submissions through search
got_search = got.search('bran' or 'brandon stark' 
                        or 'jon snow' or 'jon' #will reddit authors be included in results?
                        or 'khaleesi' or 'dany' or 'daenerys' or 'danyris', 
                        sort='comments',
                       limit=5)

#step 5 compile submission into list
title = [] 
num_comments = []
upvote_ratio = []
sub_id = []
i=0

for submission in got_search:
    i+=1
    title.append(submission.title)
    num_comments.append(submission.num_comments)
    upvote_ratio.append(submission.upvote_ratio)
    sub_id.append(submission.id)
    if i%100 == 0:
        print(f'{i} submissions completed')

#step 6 create dataframe
df_got = pd.DataFrame(
    {'title': title,
     'num_comments': num_comments,
     'upvote_ratio': upvote_ratio,
     'id':sub_id
    })
df_got

,title,num_comments,upvote_ratio,id
0,[S7E5] Post-Premiere Discussion - S7E5 'Eastwa...,26053,0.98,6tjeos
1,[S6E5] Post-Premiere Discussion - S6E5 'The Door',17604,0.97,4klpws
2,[S6E3] Post-Premiere Discussion - S6E3 'Oathbr...,11830,0.97,4ihick
3,[S6E2] Post-Premiere Discussion - S6E2 'Home',11359,0.98,4hdflw
4,[S7E5] Live Premiere Discussion - S7E5 'Eastwa...,9374,0.97,6tj3lx


In [28]:
#do same for politics 
dem_search = politics.search('kamala', 
                              sort='comments',
                             limit=5)

title = [] 
num_comments = []
upvote_ratio = []
sub_id = []
i=0

for submission in dem_search:
    i+=1
    title.append(submission.title)
    num_comments.append(submission.num_comments)
    upvote_ratio.append(submission.upvote_ratio)
    sub_id.append(submission.id) 
    if i%100 == 0:
        print(f'{i} submissions completed')

df_dem = pd.DataFrame(
    {'title': title,
     'num_comments': num_comments,
     'upvote_ratio': upvote_ratio,
     'id':sub_id
    })
df_dem

,title,num_comments,upvote_ratio,id
0,Megathread: AG Willam Barr releases his top li...,45574,0.88,b50gkr
1,Megathread: President Trump delivers remarks o...,32332,0.82,6tx8h7
2,Megathread: Likely Explosive Devices Addressed...,21359,0.90,9rlm9p
3,Megathread: President Trump announces a deal t...,12928,0.88,ajsubi
4,[Megathread] President Trump’s Address on Bord...,9081,0.91,ae2e7b


## Retrieve Comments

In [9]:
submission = reddit.submission(id=df_dem['id'][0])

In [29]:
# Instantiate list to hold comments
test_comments = []
comments_dicts = []

submission.comments.replace_more(limit=5)
for comment in submission.comments.list()[:100]:
#     print(comment.body)
    # List of comments, as strings
    test_comments.append(comment.body)

    # List of comments (dicts)
    comments_dicts.append({
        'comment': comment.body
    })
    

In [257]:
# Check 
test_comments[3]

'Holy shit anyone watching MSNBC?  This asshole sent his cult members emails asking for donations before and after the speech WTF'

In [227]:
# Put them in a dataframe, as POC
pol_df = pd.DataFrame(test_comments, columns=['comment'])
pol_df.head()
len(pol_df)

100

In [31]:
# #test to see how we'll search strings for later when we use vader
# pol_df['comment'].str.contains('joyann')
# #case sensitive
# #should write function that attributes comment to person 
# #forward slashes in links seem to operate as spaces 
# #make all comments all lowercase to simplify attributing phase

## Attribute Comment to People

In [306]:
test_dict = {'trump':["donald", 'trump', 'idiot'],
             'common_words':['it', 'its', 'the'],
            'usa':['usa','format'],
            'poppinkream':['PoppinKREAM'],
            'profanity':['shit','fucking','asshole','fuck']}

In [323]:
def attribute_comment(df, map_dict):
    """Put character's name in the character's column if it mentioned in the comment."""
    for index, row in df.iterrows(): #loop thru comments
        for key, value in map_dict.items(): #loop through dictionary
            df.loc[index,key]= int(0)
            for item in value:
                if item in row['comment']:
                    df.loc[index, key] += int(row['comment'].count(item))

In [324]:
pol_df = pd.DataFrame(test_comments, columns=['comment'])
pol_df.head(5)

,comment
0,Lawrence O'Donnell just reporting that the Tru...
1,I saw a comment that USA Today Politics ([@usa...
2,[deleted]
3,Holy shit anyone watching MSNBC? This asshole...
4,Republican on CNN saying how relieved he is th...


In [325]:
attribute_comment(pol_df, test_dict)
pol_df.tail(10)

,comment,trump,common_words,usa,poppinkream,profanity
90,Trump had two years to get his wall. Now the D...,0.0,8.0,0.0,0.0,1.0
91,I've yet to see someone tell me how the lives ...,0.0,5.0,0.0,0.0,0.0
92,Honest question: where's the gain for all the ...,0.0,5.0,0.0,0.0,0.0
93,I can't wait for Mueller to release his findin...,0.0,2.0,0.0,0.0,1.0
94,[deleted],0.0,0.0,0.0,0.0,0.0
95,The fact that conservatives still hold power i...,0.0,1.0,0.0,0.0,0.0
96,Man the shills are out in force following this...,0.0,1.0,0.0,0.0,0.0
97,"Well, media, I hope your happy. Trump said not...",0.0,5.0,0.0,0.0,0.0
98,Look at this problem I created that others won...,0.0,1.0,0.0,0.0,0.0
99,The fear mongering was out of control. Glad t...,0.0,2.0,0.0,0.0,0.0


## Add comment length column

In [291]:
pol_df['comment_length'] = ''

In [292]:
def add_comment_length(df):
    """Add the length of the comment to our dataframe (which has a column named 'comment."""
    for index, row in pol_df.iterrows():
        pol_df.loc[index,'comment_length']= len(pol_df['comment'][index]) 

In [293]:
pol_df.head()

,comment,harris,common_words,usa,poppinkream,profanity,comment_length
0,Lawrence O'Donnell just reporting that the Tru...,0.0,10.0,0.0,0.0,0.0,
1,I saw a comment that USA Today Politics ([@usa...,0.0,147.0,17.0,1.0,0.0,
2,[deleted],0.0,0.0,0.0,0.0,0.0,
3,Holy shit anyone watching MSNBC? This asshole...,0.0,2.0,0.0,0.0,2.0,
4,Republican on CNN saying how relieved he is th...,0.0,1.0,0.0,0.0,0.0,


## Add sentiment from Werlindo's file

In [32]:
# from vaderSentiment import vaderSentiment

# analyzer = vaderSentiment.SentimentIntensityAnalyzer()

# for comment in test_comments:
#     print(comment)
#     print(analyzer.polarity_scores(comment))
# #https://github.com/cjhutto/vaderSentiment#about-the-scoring 

In [316]:
# Make comments lowercase
pol_df['comment'][0].lower()

'lawrence o\'donnell just reporting that the trump 2020 campaign committee sent an email to his supporters asking them to donate to their "official secure the border fund," and that those that donated would have their name sent to trump. immediately after the speech, a second email went out again asking for the donations. \n\nproblem: the fine print says the money goes to his reelection campaign. \n\nhe held this press conference to scoop up campaign money.'